<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/button_radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import pickle
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense

with open('radar_array_train.txt', 'rb') as radar:
  x_train = pickle.load(radar)

with open('button_array_train.txt', 'rb') as button:
  y_train = pickle.load(button)

x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train[0])
print(len(x_train))
print(len(y_train))

[125.  95. 125. 475.]
512
512


In [108]:
def normalize_array_2d(array, max_value=800, new_max_value=255):
    if not isinstance(array, np.ndarray):
        raise TypeError("Input array must be a NumPy array.")
    if array.ndim != 2:
        raise ValueError("Input array must be a two-dimensional array.")
    if np.min(array) < 0:
        raise ValueError("Input array cannot contain negative values.")
    normalized_array = array / max_value
    rescaled_array = normalized_array * new_max_value
    return rescaled_array

def denormalize_array_2d(array, max_value=800, original_min_value=0):
    if not isinstance(array, np.ndarray):
        raise TypeError("Input array must be a NumPy array.")
    if array.ndim != 2:
        raise ValueError("Input array must be a two-dimensional array.")
    if original_min_value >= max_value:
        raise ValueError("Original minimum value must be less than maximum value.")
    rescaled_array = array * (max_value - original_min_value)
    denormalized_array = rescaled_array + original_min_value
    denormalized_array = denormalized_array / 255
    return denormalized_array.astype(array.dtype)

normalized_array = normalize_array_2d(x_train, max_value=800, new_max_value=255)

print(normalized_array[0])

[ 39.84375  30.28125  39.84375 151.40625]


In [125]:
import numpy as np

def convert_to_bit_array(double_array):
    bit_array = []
    for sublist in double_array:
        binary_sublist = [int(bit) for num in sublist for bit in format(int(num), '08b')]
        bit_array.append(binary_sublist)
    return bit_array

x_train_bit_array = convert_to_bit_array(normalized_array)
x_train_bit_array = np.array(x_train_bit_array)
print(x_train_bit_array[0])
print(y_train[0])
print(len(x_train_bit_array))
print(len(y_train))

[0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1]
[0 0 0 0]
512
512


In [127]:
model = Sequential()
model.add(Flatten(input_shape=(32,)))
model.add(Dense(1024, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(4, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train_bit_array, y_train, epochs=300, verbose=1)
model.save('model.keras')

Epoch 1/300
16/16 [==============================] - 1s 5ms/step - loss: 0.3218 - accuracy: 0.2324
Epoch 2/300
16/16 [==============================] - 0s 6ms/step - loss: 0.2607 - accuracy: 0.2324
Epoch 3/300
16/16 [==============================] - 0s 6ms/step - loss: 0.2522 - accuracy: 0.2324
Epoch 4/300
16/16 [==============================] - 0s 6ms/step - loss: 0.2451 - accuracy: 0.2324
Epoch 5/300
16/16 [==============================] - 0s 5ms/step - loss: 0.2383 - accuracy: 0.2324
Epoch 6/300
16/16 [==============================] - 0s 5ms/step - loss: 0.2308 - accuracy: 0.2324
Epoch 7/300
16/16 [==============================] - 0s 5ms/step - loss: 0.2208 - accuracy: 0.2324
Epoch 8/300
16/16 [==============================] - 0s 5ms/step - loss: 0.2124 - accuracy: 0.2324
Epoch 9/300
16/16 [==============================] - 0s 3ms/step - loss: 0.2033 - accuracy: 0.2422
Epoch 10/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1930 - accuracy: 0.2559
Epoch 11/

In [128]:
x_test = x_train_bit_array
y_test = y_train

model = load_model('model.keras')

predictions = model.predict(x_test, verbose=0)
print(f'Предсказание y:\n{predictions}')
print(f'Точность: {(model.evaluate(x_test, y_test, verbose=0)[1]) * 100} %')

Предсказание y:
[[2.4554156e-05 5.0384703e-04 4.2718297e-05 1.4154504e-02]
 [2.4554156e-05 5.0384703e-04 4.2718297e-05 1.4154504e-02]
 [2.4554156e-05 5.0384703e-04 4.2718297e-05 1.4154504e-02]
 ...
 [2.0942813e-05 1.6866434e-04 3.0449017e-05 8.5890908e-03]
 [2.0942813e-05 1.6866434e-04 3.0449017e-05 8.5890908e-03]
 [2.0942813e-05 1.6866434e-04 3.0449017e-05 8.5890908e-03]]
Точность: 37.3046875 %
